# GPSO optimisation example with callbacks

This notebook shows the usage of callbacks during optimisation for the better experience!

In [1]:
# imports
import os
import numpy as np
import matplotlib.pyplot as plt
from shutil import rmtree
import logging
from gpflow.utilities import print_summary

from gpso.param_space import ParameterSpace
from gpso.optimisation import GPSOptimiser, CallbackTypes, GPSOCallback
from gpso.callbacks import (
    PostIterationPlotting,
    PostUpdateLogging,
    PreFinaliseSave,
    GPFlowCheckpoints,
)
from gpso.utils import set_logger, make_dirs

plt.style.use("seaborn-poster")

We will use the same objective function as before, so no surprises there.

In [2]:
# define objective function - compare with paper Fig. 2
def obj(point, offset=0.0, rotate=True):
    x, y = point
    if rotate:
        ct = np.cos(np.pi / 4)
        st = np.sin(np.pi / 4)
        xn = ct * x + st * y
        yn = ct * y - st * x
        x = xn
        y = yn
    return (
        3 * (1 - x) ** 2.0 * np.exp(-(x ** 2) - (y + 1) ** 2)
        - 10 * (x / 5.0 - x ** 3 - y ** 5) * np.exp(-(x ** 2) - y ** 2)
        - 1 / 3 * np.exp(-((x + 1) ** 2) - y ** 2)
        - offset
    )


# bounds of the parameters we will optimise
x_bounds = [-3, 5]
y_bounds = [-3, 3]
# number of points per dimension for plotting
N_POINTS = 120

Definition of parameter space is the same as previously

In [3]:
space = ParameterSpace(
    parameter_names=["x", "y"], parameter_bounds=[x_bounds, y_bounds]
)

Now for the changes!

### Callbacks
`pyGPSO` supports so-called callbacks. They are functions which run regularly after some events. Currently there exist 5 types of callbacks:
* **post-initialise callback**: runs just once, after initialisation of the problem
* **pre-iteration callback**: runs repeatedly just before the iteration loop
* **post-iteration callback**: runs repeatedly just after the iteration loop
* **post-update callback**: runs after each update of GPR
* **pre-finalise callback**: runs just once, just before the optimisation is finished

`pyGPSO` comes with 4 already defined, hopefully useful, callbacks. However, they are very easy to create for yourself as we will see in a moment! Let's first check the predefined callbacks:

#### `PostIterationPlotting` callback
Simple - plots conditional surrogate distribution, marginal parameter distribution and ternary tree after each iteration, so you can visually check the convergence of the algorithm

In [4]:
plotting_dir = "output"
make_dirs(plotting_dir)

plotting_callback = PostIterationPlotting(
    filename_pattern=os.path.join(plotting_dir, "post_iter_plot"),
    plot_ext=".png",
    gp_mean_limits=[-10, 10],
    gp_var_limits=[0, 5],
    marginal_plot_type="kde",
    marginal_percentile=0.5,  # low percentile - just for showcase
    from_iteration=10,  # from 10th iteration, we do not care about first 4...
)

#### `PostUpdateLogging` callback
Even simpler callback - after each GPR update logs the current hyperparameters of GP regressor

In [5]:
gp_logger = PostUpdateLogging()

#### `PreFinaliseSave` callback
This callback saves parameter space (and its state, i.e. leaves) and GPR surrogate together with list of points that were either evaluated or (as centers of leaves) estimated.

In [6]:
fin_saver = PreFinaliseSave(path=plotting_dir)

#### `GPFlowCheckpoints` callback
This callback saves checkpoints of the GPR model after each update. As such it might not be useful, but can be easily altered to provide more functionality, e.g. restoring GPR model on the go.

In [7]:
checkpoints = GPFlowCheckpoints(path=plotting_dir, max_to_keep=3)

### Create your own callback!
OK, nice, you got me hooked, right? But I do not like your callbacks, I'd like to create my own!

Well, no problemo! Lets say, you want a callback that will plot surface of GPR after each GPR update. Lets start!

In [8]:
# we will subclass base callback
class GPRPlotSurface(GPSOCallback):
    # define type of callback - post-update
    callback_type = CallbackTypes.post_update

    # do all settings here!
    def __init__(self, path):
        # always super init for a sanity check and validation
        super().__init__()
        self.path = path
        # set plotting data
        x = np.linspace(x_bounds[0], x_bounds[1], N_POINTS)
        y = np.linspace(y_bounds[0], y_bounds[1], N_POINTS)
        x, y = np.meshgrid(x, y)
        self.stk = np.hstack([x.reshape((-1, 1)), y.reshape((-1, 1))])
        self.im_shape = x.shape
        self.counter = 0

    # this will be actually run when a callback is called
    # always only one argument - optimiser class itself
    def run(self, optimiser):
        # always super run for a sanity check and validation
        super().run(optimiser)
        # actual plotting
        plt.figure()
        mean, _ = optimiser.gp_surr.gpr_model.predict_y(
            optimiser.param_space.normalise_coords(self.stk)
        )
        mean = mean.numpy().reshape(self.im_shape)
        plt.imshow(
            mean, vmax=8.0, vmin=-8.0, cmap=plt.get_cmap("cividis"), origin="lower"
        )
        cbar = plt.colorbar()
        cbar.set_label("Objective-GP")
        plt.title("Surrogate surface")
        plt.savefig(os.path.join(self.path, f"surr_surface_{self.counter}.png"))
        plt.close()
        self.counter += 1


# now we need to init our brand new callback
surr_surface = GPRPlotSurface(path=plotting_dir)

### Run with callbacks
After we initialise our callbacks, we run the optimiser as usual and just pass all callbacks as a list 

In [9]:
opt = GPSOptimiser(
    parameter_space=space,
    exploration_method="tree",
    exploration_depth=5,
    update_cycle=1,
    budget=50,
    stopping_condition="evaluations",
    gp_lik_sigma=1.0e-3,
    n_workers=4,
    callbacks=[plotting_callback, gp_logger, fin_saver, checkpoints, surr_surface],
)

In [10]:
# log_level INFO: reasonable amount of information on what is happening
# log_level DEBUG: a lot of information on what is happening
set_logger(log_level=logging.INFO)
# run vanilla, with default initialisation and just 1 repetition of objective function (since it's deterministic...)
best_point = opt.run(obj)

[2020-03-20 20:09:07] INFO: Starting 2-dimensional optimisation with budget of 50 objective function evaluations...
[2020-03-20 20:09:07] INFO: Sampling 2 vertices per dimension within L1 ball of 0.25 of the domain size radius in normalised coordinates using 4 worker(s)...
[2020-03-20 20:09:07] INFO: Update step: retraining GP model and updating scores...
[2020-03-20 20:09:10] INFO: Running PostUpdateLogging callback...
[2020-03-20 20:09:10] INFO: GPR summary:
name                     class      transform         prior    trainable    shape    dtype    value
-----------------------  ---------  ----------------  -------  -----------  -------  -------  ----------------------
GPR.mean_function.c      Parameter                             True         ()       float64  -0.9939317388570423
GPR.kernel.variance      Parameter  Softplus                   True         ()       float64  6.791967471710489
GPR.kernel.lengthscales  Parameter  Softplus                   True         (1,)     float64

In [11]:
# WOHOOO, check the best point
print(best_point)
# and print summary of our trained GPR model
print_summary(opt.gp_surr.gpr_model, fmt="notebook")

GPPoint(normed_coord=array([0.23662551, 0.68518519]), score_mu=8.102201204076508, score_sigma=0.0, score_ucb=0.0, label=<PointLabels.evaluated: 1>)


name,class,transform,prior,trainable,shape,dtype,value
GPR.mean_function.c,Parameter,,,True,(),float64,0.19149976301992186
GPR.kernel.variance,Parameter,Softplus,,True,(),float64,3.163835951715155
GPR.kernel.lengthscales,Parameter,Softplus,,True,"(1,)",float64,[0.12871433]
GPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,1.0482618379448408e-06


### Aftermath

So we can see that the results are the same as before (duh), but, we can see:
1. overview table of GPR hyperparameters in the log after each update (thanks to `PostUpdateLogging` callback)
2. plots after each iteration (thanks to `PostIterationPlotting` callback)
3. saved parameter space and GPR surrogate (thanks to `PreFinaliseSave` callback)
4. checkpoints while training the GPR (courtesy of `GPFlowCheckpoints` callback)

but mainly

5. plots of surrogate mean surface after each update, so we can monitor how the training "helps" the surrogate to get the correct shape (courtesy of our brand new `GPRPlotSurface` callback)

In [12]:
# cleaning - run after you check the results!
rmtree(plotting_dir)